In [10]:
# This file is part of pydidas.
#
# Copyright 2023 - 2024, Helmholtz-Zentrum Hereon
# SPDX-License-Identifier: GPL-3.0-only
#
# pydidas is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License version 3 as
# published by the Free Software Foundation.
#
# Pydidas is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Pydidas. If not, see <http://www.gnu.org/licenses/>.

"""
The dataset module includes the Dataset subclasses of numpy.ndarray with additional
embedded metadata.
"""

__author__ = "Gudrun Lotze"
__copyright__ = "Copyright 2024, Helmholtz-Zentrum Hereon"
__license__ = "GPL-3.0-only"
__maintainer__ = "Gudrun Lotze"
__status__ = "Development"

In [11]:
import os
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
import matplotlib.cm as cm
from scipy.optimize import curve_fit
from pydidas.core import Dataset
from pydidas.data_io import import_data
%matplotlib inline


import importlib
import sys

fname = '/Users/lotzegud/pydidas/pydidas_plugins/proc_plugins/stress_strain.py'
spec = importlib.util.spec_from_file_location('stress_strain', fname)
stress_strain = importlib.util.module_from_spec(spec)
spec.loader.exec_module(stress_strain)

# Check if the module object is created
if hasattr(stress_strain, '__file__'):
    print("Module was successfully loaded.")

# Inspect module attributes
print("Module attributes:", dir(stress_strain))

#We need to append the path to the sys.path to import the module
sys.path.append('/Users/lotzegud/pydidas/pydidas_plugins/proc_plugins')

from  stress_strain import extract_d_spacing, chi_pos_verification, extract_d_spacing, ds_slicing, combine_sort_d_spacing_pos_neg, group_d_spacing_by_chi, idx_s2c_grouping

import pydidas 
print('Pydidas version: ', pydidas.version)

Module was successfully loaded.
Module attributes: ['Dataset', '__author__', '__builtins__', '__cached__', '__copyright__', '__doc__', '__file__', '__license__', '__loader__', '__maintainer__', '__name__', '__package__', '__spec__', '__status__', 'chi_pos_verification', 'combine_sort_d_spacing_pos_neg', 'connected_components', 'csr_matrix', 'ds_slicing', 'extract_d_spacing', 'group_d_spacing_by_chi', 'h5', 'idx_s2c_grouping', 'import_data', 'np', 'os', 'pre_regression_calculation']
Pydidas version:  24.06.05


Auxiliary functions

In [23]:
def chi_gen(chi_start, chi_stop, delta_chi):
    if chi_start >= chi_stop:
        raise ValueError('chi_start has to be smaller than chi_stop')
    return np.arange(chi_start, chi_stop, delta_chi)


def predefined_metric_calculation(metric_name, chi, x, y, d0, spatial_var, phase_shift):
    """ Calculate predefined metric based on name, applying spatial variation even if x is not provided. """
    # Handle spatial variation by introducing a default or random x if none is provided
    if x is None and spatial_var:
        x = np.random.uniform(0, 1)  #A random x between 0 and 5
    if metric_name == "position":
        return 0.2832*np.sin(np.deg2rad(chi+phase_shift))**2 + d0 + (0.01 * x if spatial_var else 0)
    if metric_name == "area":
        return np.random.uniform(6, 37, size=len(chi)) + 0.1 * y
    if metric_name == "FWHM":
        return np.random.uniform(0.35, 0.75, size=len(chi))
    if metric_name == "background at peak":
        return np.random.uniform(2.3, 5.3, size=len(chi))
    if metric_name == "total count intensity":
        return np.random.uniform(80, 800, size=len(chi))
    return np.random.uniform(1.5708, 3.141, size=len(chi))  # Fallback for unknown metrics

def generate_spatial_fit_res(y_range, x_range=None, chi_start=-175, chi_stop=180, delta_chi=10, fit_labels=None, spatial_var=True, phase_shift=0):
    '''
    chi [degree]
    phase_shift [degree]
    '''
    
    if fit_labels is None:
        fit_labels = '0: position; 1: area; 2: FWHM; 3: background at peak; 4: total count intensity'
    fit_labels_dict = {int(k.split(':')[0].strip()): k.split(':')[1].strip() for k in fit_labels.replace(', ', ';').split(';')}

    chi = chi_gen(chi_start, chi_stop, delta_chi)
    d0 = 25  # in nm

    # Determine the dimensions based on x_range
    if x_range is not None:
        result_array = np.empty((len(y_range), len(x_range), len(chi), len(fit_labels_dict)))
    else:
        result_array = np.empty((len(y_range), len(chi), len(fit_labels_dict)))
        x_range = [None]  # Simulate the absence of x values

    # Perform calculations for each y and x, and across all metrics
    for j, y in enumerate(y_range):
        for i, x in enumerate(x_range):
            fit_results = []
            for idx in sorted(fit_labels_dict.keys()):
                metric_name = fit_labels_dict[idx]
                result = predefined_metric_calculation(metric_name, chi, x, y, d0, spatial_var, phase_shift)
                fit_results.append(result)

            fit_results = np.array(fit_results)
            # Adjust how results are stored based on the presence of x_range
            # Debug print statements
            #print(f"fit_results.T.shape: {fit_results.T.shape}, j: {j}, i: {i}")
            #print('x_range:', x_range)
            if x is not None:
                result_array[j, i, :, :] = fit_results.T
            else:
                result_array[j, :, :] = fit_results.T  # Ensure dimensionality matches expected (len(chi), len(fit_labels_dict))

    return result_array

def adding_noise_d_spacing(d_spacing, scale=0.03, mean_value=1, seed=10):
    '''
    ds: Dataset, expecting in first column d-spacing values.
    Noise follows a Laplace distribution
    mean_value, scale: Mean_value and scale parameter of Laplace distribution
    Seed: radnom seed
    
    '''
    ds = d_spacing.copy()
    #Introducing seed and random noise for d_spacing
    # Define the scale parameter for the Laplace distribution
    # Generate Laplace noise centered around the mean value
    d_spacing_noise = np.random.default_rng(seed=seed).laplace(mean_value, scale, size=d_spacing.array.shape[0])
 
    d_spacing_with_noise = d_spacing.array[:,0] + d_spacing_noise
    #update array in ds
    ds.array[:,0] = d_spacing_with_noise
    return ds
  
def extract_units(ds):
    """
    Extract units from data_label in a Pydidas.Dataset, corresponding to parameters in fit_labels.

    Parameters:
    ds (pydidas.Dataset): The dataset containing fit_labels and data_label.

    Returns:
    dict: A dictionary mapping fit_labels to their corresponding units.

    Raises:
    TypeError: If ds is not an instance of pydidas.Dataset.
    ValueError: If a unit for any fit label is not found.
    """
        
    # Ensure ds is an instance of Dataset
    if not isinstance(ds, Dataset):
        raise TypeError("Input must be an instance of pydidas.Dataset")
       
    
    chi_key, (pos_key, _) = chi_pos_verification(ds)
    print(ds.axis_units[chi_key])
        
    data_label=ds.data_label
    fit_labels=ds.axis_labels[pos_key]
    
    # Step 1: Extract parameter names from fit_labels using dictionary comprehension
    fit_labels_dict = {int(item.split(":")[0].strip()): item.split(":")[1].strip() for item in fit_labels.split(";")}

    # Step 2: Extract units from data_label
    data_label_dict = {}
    data_label_parts = data_label.split(';')
    for part in data_label_parts:
        if '/' in part:
            name, unit = part.split('/')
            name = name.split(':')[-1].strip()
            unit = unit.strip()
            data_label_dict[name] = unit
    
    # Alternative approach using dictionary comprehension        
    ## Step 2: Extract units from data_label using dictionary comprehension
    #data_label_dict = {
    #    part.split(':')[-1].split('/')[0].strip(): part.split('/')[-1].strip()
    #    for part in data_label.split(';') if '/' in part
    #}

    # Step 3: Create a mapping of fit_labels to their corresponding units
    #result = {param: data_label_dict.get(param, 'Unit not found') for #param in fit_labels_dict.values()}
    result = {}
    for param in fit_labels_dict.values():
        print(param, unit)
        try:
            unit = data_label_dict[param]
        except KeyError:
            raise ValueError(f"Unit not found for parameter: {param}")
        result[param] = unit
          
    # Append the unit for chi
    try:
        chi_unit = ds.axis_units[chi_key]
        if not chi_unit:
            raise ValueError("Unit for chi is empty.")
        result['chi'] = chi_unit
    except KeyError:
        raise ValueError("Unit not found for chi.")
    
    return result

def chi_pos_unit_verification(ds):
    """
    Verify that the units for chi and position are correct.
    
    Parameters:
    ds (Dataset): A Pydidas Dataset with fitting results. Position contains d_spacing values
    
    Returns:
    bool: True if the units are correct, False otherwise.

    Raises:
    TypeError:  If ds is not an instance of pydidas.Dataset.
    ValueError: If the units for chi or position are not allowed.
  
    """
    
    
    if not isinstance(ds, Dataset):
        raise TypeError("Input must be an instance of pydidas.Dataset")
    
    ds_units = extract_units(ds)
    
    #position/pos contains the units for d_spacing
    pos_units_allowed = ['nm', 'A']
    chi_units_allowed = ['deg', 'rad']
    
    units_to_check = ['position', 'chi']
    
    for item, val in ds_units.items():
        if item in units_to_check:
            if item == 'position':
                if val not in pos_units_allowed:
                    raise ValueError(f"Unit {val} not allowed for {item}.")
            if item == 'chi':
                if val not in chi_units_allowed:
                    raise ValueError(f"Unit {val} not allowed for {item}.")
    
    return True




def plot_d_spacing_vs_chi(ds, chi, positions):
    """
    Plots d_spacing vs chi for specified (x, y) positions in the result array using Matplotlib's OOP interface.

    Parameters:
        ds (Dataset): The 4D array containing measurement data.
        chi (numpy.ndarray): The array of chi values.
        positions (list of tuples): A list of (x_index, y_index) tuples specifying the positions to plot.
    """

    print('ds.shape:', ds.shape)
    
    fig, ax = plt.subplots()
    for (x_index, y_index) in positions:
        # Extract d_spacing for the specific position
        d_spacing = ds.array[x_index, y_index, :, 0]  # d_spacing is the first property in the last dimension
        
        # Plotting using the axes object
        ax.plot(chi, d_spacing, label=f'(x={x_index}, y={y_index})', marker='o', linestyle='--')
    
    ax.set_xlabel('chi [deg]')
    ax.set_ylabel('d_spacing')
    ax.set_title('d_spacing vs chi for various x,y')
    ax.grid(True)
    
    ax.legend()
    plt.show()
    

    

def main():
    
    
    #Creating of dummy data
    delta_chi =10
    chi_start= -180
    chi_stop= 181
    phase_shift = 70
    chi=chi_gen(chi_start, chi_stop, delta_chi)
    print('Chi\n', chi)
   
    #x, y in um
    y = np.arange(2, 8)
    x = np.arange(0, 5)

    #labels
    fit_labels= '0: position; 1: area; 2: FWHM; 3: background at peak; 4: total count intensity'
    fit_labels_dict = {int(item.split(":")[0].strip()): item.split(":")[1].strip() for item in fit_labels.split(";")}
    num_labels = len(fit_labels_dict)
    
    data_labels= 'position / nm^-1; area / (cts * nm^-1); FWHM / nm^-1;background at peak / cts; total count intensity / cts'
    
    #creation of Pydidas Dataset
    axis_labels= ['y', 'x', 'chi', fit_labels]
    axis_ranges = {0: y, 1:x, 2: chi , 3: np.arange(num_labels)} 
    axis_units= ['um', 'um', 'deg', '']    
   
    result_array_spatial = generate_spatial_fit_res(y, x, chi_start,chi_stop, delta_chi, fit_labels , spatial_var=True, phase_shift=phase_shift)
    #result_array = generate_spatial_fit_res(y, x, chi_start,chi_stop, delta_chi, fit_labels , spatial_var=False)
    print('Result array spatial shape\n', result_array_spatial.shape)
    

    ds = Dataset(result_array_spatial,  axis_labels=axis_labels, axis_units=axis_units,axis_ranges=axis_ranges, data_label=data_labels)
    #ds_no_spatial = Dataset(result_array,  axis_labels=axis_labels, axis_ranges=axis_ranges)
    
    print(40*"\N{popcorn}")
    print(ds)
    print(40*"\N{popcorn}")
    ds_units = extract_units(ds)
    print(ds_units)
    print(40*"\N{popcorn}")
    
    axis_labels = ds.axis_labels
    print('Axis labels\n', axis_labels)
    print('Shape of dataset\n', ds.shape)
    print('Type ds\n', type(ds))
    print(40*"\N{popcorn}")
    
    
    #visualisation
    #positions = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
    #plot_d_spacing_vs_chi(ds, chi, positions)
    #plot_d_spacing_vs_chi(ds_no_spatial, chi, positions)
      
    #choose random location in ds
    x_idx = 5
    y_idx = 3
    
    #plot_d_spacing_vs_chi(ds, chi, [(x_idx, y_idx)])
    
    
    # slice Dataset based on location
    ds1 = ds[x_idx, y_idx]
    print('Content of ds1 before noise addition\n', ds1.array[:,0])
          
    #Introducing seed and random noise for d_spacing
    ds1 = adding_noise_d_spacing(ds1, scale=0.01, mean_value=1, seed=10)
    #print('Shape of ds1 dataset\n', ds1.shape)
    #print('Content ds1 array after noise addition\n',  ds1.array[:,0])
    #print(ds1)
    
    
    #Introducing np.nan 
    #print('Content ds1 array',  ds1.array[:,0])
    #ds1.array[0:9:2,0] = np.nan
    #ds1.array[18:28:2,0] = np.nan
   
    fig, ax = plt.subplots()
    ax.plot(chi, ds1.array[:,0], linestyle='None', marker='*', label='d_spacing', markersize=9)
    ax.grid()
    ax.set_ylabel('d [nm]')
    ax.set_xlabel('chi [deg]')
    ax.set_title('d_spacing vs. chi')
    fig.show()
          
    chi_key, (pos_key, pos_idx) = chi_pos_verification(ds1)
    print('chi_key, pos_key, pos_idx:')
    print(chi_key, pos_key, pos_idx)
    if chi_pos_unit_verification(ds1):
        chi, d_spacing = ds_slicing(ds1)    
        
    d_spacing_pos, d_spacing_neg=group_d_spacing_by_chi(d_spacing, chi)
    
    fig, ax = plt.subplots()
    ax.plot(np.sin(np.deg2rad(chi))**2, d_spacing, linestyle='None', marker='o', label='d_spacing', markerfacecolor='None', markersize=9)
    ax.plot(d_spacing_pos.axis_ranges[0], d_spacing_pos.array, linestyle='None', marker='X', label='d_spacing_pos', markersize=7)
    ax.plot(d_spacing_neg.axis_ranges[0], d_spacing_neg.array, linestyle='None', marker='*', label='d_spacing_neg', markersize=7)
    ax.set_ylabel('d [nm]')
    ax.set_xlabel('sin^2(chi)')
    ax.set_title('sin^2(chi) vs d_spacing')
    interval = 4
    x_index = np.arange(0, len(chi), interval)
    ax2top=ax.twiny()
    ax2top.set_xlim(ax.get_xlim())
    formatted_labels = [f'{val:.2f}' for val in chi[x_index]]
    ax2top.set_xticks(x_index)
    ax2top.set_xticklabels(formatted_labels)
    ax2top.set_xlabel('chi [deg]')
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.03))
    ax.grid()
    fig.show()
    
      
    
    print('s2c_pos', d_spacing_pos.axis_ranges[0])
    print('d_spacing_pos', d_spacing_pos.array)
    print('s2c_neg', d_spacing_neg.axis_ranges[0])
    print('d_spacing_neg', d_spacing_neg.array)
    
    d_spacing_combined = combine_sort_d_spacing_pos_neg(d_spacing_pos, d_spacing_neg)
    #print('d_spacing_combined, shape', d_spacing_combined.shape)
    #print('d_spacing_combined', d_spacing_combined) 
    
    d_diff= np.diff(d_spacing_combined.array, axis=0).squeeze()
    x_diff = np.sin(2*np.arcsin(np.sqrt(d_spacing_combined.axis_ranges[1])))
    
    print('d_diff', d_diff)
    print('x_diff', x_diff)
    print(d_spacing_combined.axis_ranges[1])
    
    fig, ax = plt.subplots()
    ax.plot(x_diff, d_diff, linestyle='None', marker='o')
    ax.set_title('d(+) - d(-) vs sin(2*chi)')
    ax.set_xlabel('sin(2*chi)')
    ax.set_ylabel('d(+) - d(-)')
    ax.grid()
    fig.show()
    
    
    ds2 = ds1.copy()
    print(ds2)
    print(ds2+2)
    
    
    def pre_regression_calculation(d_spacing_combined):
        ''' 

        Prepares data for regression analysis based on d-spacing values.

        Parameters:
        d_spacing_combined (Dataset): Pydidas Dataset with d-spacing values vs sin^2(chi) values.
            - Shape (2, N) where d_spacing_combined[0, :] represents d(-) values and
            d_spacing_combined[1, :] represents d(+) values.
            - If a value in either d(+) or d(-) is missing (np.nan), it is not taken into account for the calculations.

        Returns:
        d_spacing_avg (Dataset): Pydidas Dataset containing the average of (d(+) + d(-))/2 vs sin^2(chi).
            - axis_ranges[0] corresponds to sin^2(chi).
            - axis_labels[0] is 'sin^2(chi)'.
            - data_label is 'd_spacing_mean'.

        d_spacing_diff (Dataset): Pydidas Dataset containing the difference of d(+) - d(-) vs sin(2*chi).
            - axis_ranges[0] corresponds to sin(2*chi).
            - axis_labels[0] is 'sin(2*chi)'.
            - data_label is 'd_spacing_diff'.
     
        '''
        # Check if d_spacing_combined is an instance of Dataset
        if not isinstance(d_spacing_combined, Dataset):
            raise TypeError("Input d_spacing_combined must be an instance of Dataset.")

        
        #d_spacing_combined[0,5] = np.nan
        # This is the case where one part of the d_spacing pair is missing and not taken into account for the average
        d_spacing_avg= np.mean(d_spacing_combined, axis=0)
        d_spacing_avg.axis_ranges={0: d_spacing_combined.axis_ranges[1]}
        d_spacing_avg.axis_labels={0: 'sin^2(chi)'}
        d_spacing_avg.data_label='d_spacing_mean'
        
        
        #d-, d+
        #d[1,1]-d[0,1]
        #vs sin(2*chi)
        d_spacing_diff= np.diff(d_spacing_combined, axis=0).squeeze()
        d_spacing_diff.data_label='d_spacing_diff'
        d_spacing_diff.axis_labels={0: 'sin(2*chi)'} #or #TODO 'sin(2chi)' or 'sin_2chi'
        #calculation of sin(2*chi) from sin^2(chi)
        d_spacing_diff.axis_ranges ={0: np.sin(2*np.arcsin(np.sqrt(d_spacing_combined.axis_ranges[1])))}
        return d_spacing_avg, d_spacing_diff
    
    def plot_pre_regression_calculation_res(d_spacing_combined, d_spacing_avg, d_spacing_diff):
        import matplotlib.gridspec as gridspec
        # Create a GridSpec with explicit height ratios
        fig = plt.figure(figsize=(6, 8))  # Adjust figure size as needed
        gs = gridspec.GridSpec(nrows=2, ncols=1, height_ratios=[1, 1], hspace=0.5)  # Increase hspace for more vertical space
    
        # Create the subplots using ax[0] and ax[1]
        ax = [None, None]
        ax[0] = fig.add_subplot(gs[0])
        ax[1] = fig.add_subplot(gs[1])
        
        ax[0].plot(d_spacing_avg.axis_ranges[0],  d_spacing_avg.array, linestyle='None', marker='o', label='d_spacing_avg.data_label', markerfacecolor='None', markersize=9)
        ax[0].plot(d_spacing_combined.axis_ranges[1], d_spacing_combined.array[1,:], linestyle='None', marker='X', label='d_spacing_pos', markersize=7)
        ax[0].plot(d_spacing_combined.axis_ranges[1],  d_spacing_combined.array[0,:], linestyle='None', marker='*', label='d_spacing_neg', markersize=7)
        ax[0].set_ylabel('d [nm]')
        ax[0].set_xlabel(d_spacing_avg.axis_labels[0])
        ax[0].set_title('sin^2(chi) vs d_spacing')
        interval = 4
        x_index = np.arange(0, len(chi), interval)
        ax2top=ax[0].twiny()
        ax2top.set_xlim(ax[0].get_xlim())
        formatted_labels = [f'{val:.2f}' for val in chi[x_index]]
        ax2top.set_xticks(x_index)
        ax2top.set_xticklabels(formatted_labels)
        ax2top.set_xlabel('chi [deg]')
        ax[0].legend(loc='upper right', bbox_to_anchor=(1.425, 1.03))
        
        ax[0].grid()
        
        ax[1].plot(d_spacing_diff.axis_ranges[0], d_spacing_diff.array, linestyle='None', marker='o', label=d_spacing_diff.data_label, markersize=9)
        ax[1].set_title('d(+) - d(-) vs sin(2*chi)')
        ax[1].set_xlabel(d_spacing_diff.axis_labels[0])
        ax[1].set_ylabel(d_spacing_diff.data_label)
        ax[1].grid()
        # Adjust the space between the subplots
        fig.subplots_adjust(hspace=0.2)  # Adjust the vertical space    
        fig.show() 
        
    d_spacing_avg, d_spacing_diff = pre_regression_calculation(d_spacing_combined) 
    plot_pre_regression_calculation_res(d_spacing_combined, d_spacing_avg, d_spacing_diff)
                
        

if __name__ == "__main__":
    main()

  fig.show()



Chi
 [-180 -170 -160 -150 -140 -130 -120 -110 -100  -90  -80  -70  -60  -50
  -40  -30  -20  -10    0   10   20   30   40   50   60   70   80   90
  100  110  120  130  140  150  160  170  180]
Result array spatial shape
 (6, 5, 37, 5)
🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿🍿
Dataset(
axis_labels: {
    0: 'y'
    1: 'x'
    2: 'chi'
    3: '0: position; 1: area; 2: FWHM; 3: background at peak; 4: total count
       intensity'},
axis_ranges: {
    0: array([2, 3, 4, 5, 6, 7])
    1: array([0, 1, 2, 3, 4])
    2: array([-180, -170, ...,  170,  180])
    3: array([0, 1, 2, 3, 4])},
axis_units: {
    0: 'um'
    1: 'um'
    2: 'deg'
    3: ''},
metadata: {},
data_unit: ,
data_label: position / nm^-1; area / (cts * nm^-1); FWHM / nm^-1;background at peak / cts; total count intensity / cts,
array([[[[ 25.250072,  33.347939, ...,   4.581937, 116.154995],
         [ 25.27466 ,  26.051261, ...,   4.583039, 260.707988],
         ...,
         [ 25.2124  ,  19.908757, ...,   3.467339, 505.916816

ValueError: Unit nm^-1 not allowed for position.

In [ ]:
import sys
sys.platform
sys.path

In [ ]:
import pydidas

In [ ]:
print(pydidas)
print(pydidas.version)

In [19]:
import os
from pydidas.data_io.import_export import import_data

fname='/Users/lotzegud/pydidas_training/Peak_1_2/node_03_peak_1.h5'

ds= import_data(fname)
print(ds)

Dataset(
axis_labels: {
    0: 'y'
    1: 'x'
    2: 'chi'
    3: '0: position; 1: area; 2: FWHM; 3: background at peak; 4: total count
       intensity'},
axis_ranges: {
    0: array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
              6.5, 7. , 7.5, 8. , 8.5])
    1: array([ 0. ,  0.5, ...,  9.5, 10. ])
    2: array([-174.987382, -164.987736, ...,  165.000559,  175.000204])
    3: array([0, 1, 2, 3, 4])},
axis_units: {
    0: 'um'
    1: 'um'
    2: 'deg'
    3: ''},
metadata: {'indices': '[]', 'dataset': 'entry/data/data'},
data_unit: ,
data_label: position / nm^-1; area / (cts * nm^-1); FWHM / nm^-1; background at peak / cts; total intensity / cts,
array([[[[ 26.081963,   8.901319, ...,   3.462794, 197.2645  ],
         [ 26.014532,  15.422322, ...,   3.736583, 341.77817 ],
         ...,
         [ 26.027718,  10.280365, ...,   3.058779, 227.82591 ],
         [ 26.086248,   6.569885, ...,   3.303417, 145.59697 ]],

        [[ 26.089058,   8.715827, ...,  